In [88]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [118]:
"""
Implement a decision tree for classification. Let’s imagine we want 
to predict rain (1) and no-rain (0) for a given day. We have two predictors:

    x1 is weather type (0 = partly cloudy, 1 = cloudy, 2 = sunny)
    x2 is atmospheric pressure (0 = low, 1 = high)
"""

x1 = [0, 1, 1, 2, 2, 2]
x2 = [0, 0, 1, 1, 1, 0]

y=np.asarray([0,0,0,1,1,0])


# a=x1

# partition = {c: (a==c).nonzero()[0] for c in np.unique(a)}
# print partition


In [119]:
def partition(a):
    return {c: (a==c).nonzero()[0] for c in np.unique(a)}

In [120]:

def entropy(s):
    res = 0
    val, counts = np.unique(s, return_counts=True)
    #print val,counts
    freqs = counts.astype('float')/len(s)
    #print freqs
    for p in freqs:
        if p != 0.0:
            res -= p * np.log2(p)
    return res

en1 = entropy(y)
print en1

0.918295834054


In [121]:
"""Mutual information is the difference between the entropy
of the unsplitted set, and the average of the entropy of each split, 
weighted by the number of elements in the subset.
"""

def mutual_information(y, x):
    

    res = entropy(y)

    # We partition x, according to attribute values x_i
    val, counts = np.unique(x, return_counts=True)
#     print val,counts
    freqs = counts.astype('float')/len(x)
#     print freqs
#     print x[:]
    # We calculate a weighted average of the entropy
    for p, v in zip(freqs, val):
        #print p,v,x
        #print x==v
        #print y[x==v]
        res -= p * entropy(y[x == v]) 

    return res
"""---------------------------------------------------------------------"""

X = np.array([x1, x2]).T
print X

for x_attr in X.T:
    print x_attr

gain = mutual_information(y, [0 ,1 ,1 ,2 ,2,2]) 
print gain

gains=np.array([ mutual_information(y, x_attr) for x_attr in  X.T])
print gains.shape,gains


# x=[0,1,1,2,2,2]
# for v,p in zip([0,1,2],[0.16,0.33,0.5]):
#     print v
#     print entropy(y[x == v])

#y=[0,0,0,1,1,0]

[[0 0]
 [1 0]
 [1 1]
 [2 1]
 [2 1]
 [2 0]]
[0 1 1 2 2 2]
[0 0 1 1 1 0]
0.459147917027
(2,) [ 0.45914792  0.45914792]


In [122]:
from pprint import pprint

def is_pure(s):
    return len(set(s)) == 1

def recursive_split(x, y):
    # If there could be no split, just return the original set
    if is_pure(y) or len(y) == 0:
        return y

    # We get attribute that gives the highest mutual information
    gain = np.array([mutual_information(y, x_attr) for x_attr in x.T])
    selected_attr = np.argmax(gain)
#     print gain
#     print selected_attr

    # If there's no gain at all, nothing has to be done, just return the original set
    if np.all(gain < 1e-6):
        return y


    # We split using the selected attribute
    print x[:, selected_attr].shape,x[:,selected_attr]
    sets = partition(x[:, selected_attr])
    print sets

    res = {}
    for k, v in sets.items():
        y_subset = y.take(v, axis=0)
        x_subset = x.take(v, axis=0)
        
        res["x_%d = %d" % (selected_attr, k)] = recursive_split(x_subset, y_subset)

    return res

X = np.array([x1, x2]).T
pprint(recursive_split(X, y))

(6,) [0 1 1 2 2 2]
{0: array([0]), 1: array([1, 2]), 2: array([3, 4, 5])}
(3,) [1 1 0]
{0: array([2]), 1: array([0, 1])}
{'x_0 = 0': array([0]),
 'x_0 = 1': array([0, 0]),
 'x_0 = 2': {'x_1 = 0': array([0]), 'x_1 = 1': array([1, 1])}}
